In [55]:
import torch
import numpy as np
import time
import math
import matplotlib.pyplot as plt

## GPU

Jednym z kluczowych udogodnień PyTorcha jest możliwość szybkiego i prostego przenoszenia obliczeń między CPU a GPU.

In [56]:
torch.cuda.is_available()

False

In [57]:
device = torch.device("cuda") 
device

device(type='cuda')

In [58]:
x = torch.zeros(2, 3)
print("X", x)
x = x.to(device)
print("X", x)

X tensor([[0., 0., 0.],
        [0., 0., 0.]])


AssertionError: Torch not compiled with CUDA enabled

#### Co nam to daje?

In [59]:
x = torch.randn(5000, 5000)

## CPU version
start_time = time.time()
_ = torch.matmul(x, x)
end_time = time.time()
print(f"CPU time: {(end_time - start_time):6.5f}s")

## GPU version
x = x.to(device)
# The first operation on a CUDA device can be slow as it has to establish a CPU-GPU communication first. 
# Hence, we run an arbitrary command first without timing it for a fair comparison.
_ = torch.matmul(x*0.0, x)
start_time = time.time()
_ = torch.matmul(x, x)
end_time = time.time()
print(f"GPU time: {(end_time - start_time):6.5f}s")

CPU time: 0.57909s


AssertionError: Torch not compiled with CUDA enabled

#### Ciąg komend do zapisania
Losowość jest bardzo ważnym elementem trenowania sieci neuronowych (o czym się wkrótce przekonamy). Eksperymentując, chcielibyśmy jednak, żeby przy każdym uruchomieniu programu, wynik był taki sam. Tylko w takim przypadku jesteśmy w stanie coś wnioskować. Do zapewnienia możliwości reprodukcji wyników na GPU za pomocą pytorcha służa poniższe komendy:

In [ ]:
# GPU operations have a separate seed we also want to set
if torch.cuda.is_available(): 
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    
# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

## Tworzenie modeli
Tworzymy tensory z parametrami, wagi i biasy, obliczamy gradienty, aktualizujemy parametry itd.
lub
wykorzystujemy torch.nn

W torch.nn możemy znaleźć wiele przydatnych predefiniowanych klas jak wagi sieci, funkcje aktywacji, funkcje straty, optymalizatory itp. Jeszcze raz, dokumentacja jest fajna: https://pytorch.org/docs/stable/nn.html

In [60]:
import torch.nn as nn

Dodatkowo, funkcje wykorzystywane do budowania sieci znajdują się w module torch.nn.functional

In [61]:
import torch.nn.functional as F

### nn.Module
W PyTorchu, sieci neuronowe są zbudowane z modułów - obiektów klas dziedziczących po klasie nn.Module. Moduły mogą składać się z innych modułów, a sieć neuronowa w całości też zazwyczaj jest modułem.

Podstawowy template dla modułu wygląda następująco:

In [62]:
class MyModule(nn.Module):
    
    def __init__(self):
        super().__init__()
        # Some init for my module
        
    def forward(self, x):
        # Function for performing the calculation of the module.
        pass

W konstruktorze modułu zazwyczaj inicjowane są wszystkie parametry, lub wewnętrzne moduły (np. warstwy sieci).

W funkcji forward zapisane są wszystkie obliczenia, które są wykonywane kiedy wywołujemy dany moduł. Przykładowo: (nn = MyModule(); nn(x)). 

Kalkulacja wstecz (backwards) jest tworzona automatycznie, ale w razie potrzeby można ją też nadpisywać.

# Stwórzmy pierwsza sieć neuronową 

In [63]:
class SimpleClassifier(nn.Module):

    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()
        # Initialize the modules we need to build the network
        self.linear1 = nn.Linear(num_inputs, num_hidden)
        self.act_fn = nn.Tanh()
        self.linear2 = nn.Linear(num_hidden, num_outputs)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.linear1(x)
        x = self.act_fn(x)
        x = self.linear2(x)

        return x

Wypisywanie zawartości modelu:

In [64]:
model = SimpleClassifier(num_inputs=2, num_hidden=4, num_outputs=1)
# Printing a module shows all its submodules
print(model)

SimpleClassifier(
  (linear1): Linear(in_features=2, out_features=4, bias=True)
  (act_fn): Tanh()
  (linear2): Linear(in_features=4, out_features=1, bias=True)
)


Co tak właściwie mamy w środku?

In [65]:
for name, param in model.named_parameters():
    print(f"Parameter {name}, shape {param.shape}")

Parameter linear1.weight, shape torch.Size([4, 2])
Parameter linear1.bias, shape torch.Size([4])
Parameter linear2.weight, shape torch.Size([1, 4])
Parameter linear2.bias, shape torch.Size([1])


## Activation functions

In [66]:
act_fn_by_name = {
    "sigmoid": torch.sigmoid,
    "tanh": torch.tanh,
    "relu": F.relu,
    "leakyrelu": F.leaky_relu
}

In [67]:
def get_grads(act_fn, x):
    x = x.clone().requires_grad_() # Mark the input as tensor for which we want to store gradients
    out = act_fn(x)
    out.sum().backward() # Summing results in an equal gradient flow to each element in x
    return x.grad # Accessing the gradients of x by "x.grad"

In [68]:
# %matplotlib inline
def vis_act_fn(act_fn, name, ax, x):
    # Run activation function
    y = act_fn(x)
    y_grads = get_grads(act_fn, x)
    # Push x, y and gradients back to cpu for plotting
    x, y, y_grads = x.cpu().numpy(), y.cpu().numpy(), y_grads.cpu().numpy()
    ## Plotting
    ax.plot(x, y, linewidth=2, label="ActFn")
    ax.plot(x, y_grads, linewidth=2, label="Gradient")
    ax.set_title(name)
    ax.legend()
    ax.set_ylim(-1.5, x.max())

x = torch.linspace(-5, 5, 1000) # Range on which we want to visualize the activation functions
## Plotting
rows = math.ceil(len(act_fn_by_name)/2.0)
fig, ax = plt.subplots(rows, 2, figsize=(8, rows*4))
for i, name in enumerate(act_fn_by_name.keys()):
    vis_act_fn(act_fn_by_name[name], name, ax[divmod(i,2)], x)
fig.subplots_adjust(hspace=0.3)
plt.show()
plt.close()

<IPython.core.display.Javascript object>

# Dane
PyTorch posiada zaimplementowanych kilka funkcjonalności, które pozwalają wczytywać i manipulować danymi. Znajdują się one w pakiecie torch.utils.data.

In [69]:
import torch.utils.data as data

Główną strukturą, w ramach której przechowywane (lub niekoniecznie) są dane jest klasa Dataset. Stwórzmy sobie przykładowy zbiór danych - tzw. ciągły XOR. Przykład zaczerpnięty z https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial2/Introduction_to_PyTorch.html#Learning-by-example:-Continuous-XOR

In [70]:
class XORDataset(data.Dataset):

    def __init__(self, size, std=0.1):
        """
        Inputs:
            size - Number of data points we want to generate
            std - Standard deviation of the noise (see generate_continuous_xor function)
        """
        super().__init__()
        self.size = size
        self.std = std
        self.generate_continuous_xor()

    def generate_continuous_xor(self):
        # Each data point in the XOR dataset has two variables, x and y, that can be either 0 or 1
        # The label is their XOR combination, i.e. 1 if only x or only y is 1 while the other is 0.
        # If x=y, the label is 0.
        data = torch.randint(low=0, high=2, size=(self.size, 2), dtype=torch.float32)
        label = (data.sum(dim=1) == 1).to(torch.long)
        # To make it slightly more challenging, we add a bit of gaussian noise to the data points.
        data += self.std * torch.randn(data.shape)

        self.data = data
        self.label = label

    def __len__(self):
        # Number of data point we have. Alternatively self.data.shape[0], or self.label.shape[0]
        return self.size

    def __getitem__(self, idx):
        # Return the idx-th data point of the dataset
        # If we have multiple things to return (data point and label), we can return them as tuple
        data_point = self.data[idx]
        data_label = self.label[idx]
        return data_point, data_label

In [71]:
dataset = XORDataset(size=200)
print("Size of dataset:", len(dataset))
print("Data point 0:", dataset[0])

Size of dataset: 200
Data point 0: (tensor([0.0177, 0.0460]), tensor(0))


In [72]:
examples = dataset.data
labels = dataset.label
data_0 = examples[labels == 0]
data_1 = examples[labels == 1]

plt.figure(figsize=(4,4))
plt.scatter(data_0[:,0], data_0[:,1], edgecolor="#333", label="Class 0")
plt.scatter(data_1[:,0], data_1[:,1], edgecolor="#333", label="Class 1")
plt.title("Dataset samples")
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

## Data loaders (Ładowacze danych?!)

In [73]:
data_loader = data.DataLoader(dataset,batch_size=8, shuffle=True)

In [74]:
next(iter(data_loader))

[tensor([[ 1.0084,  0.0747],
         [ 0.9814,  0.9117],
         [ 0.1002,  0.0186],
         [ 0.1190,  1.2851],
         [ 0.1337, -0.0595],
         [ 0.9620,  1.0416],
         [ 0.8841, -0.0891],
         [-0.1582,  0.8514]]),
 tensor([1, 0, 0, 1, 0, 0, 1, 1])]

In [75]:
len(data_loader)

25

In [76]:
next(iter(data_loader))[0].shape[0]*len(data_loader)

200

In [77]:
data_loader_2 = data.DataLoader(dataset,batch_size=7, shuffle=True)
next(iter(data_loader_2))[0].shape[0]*len(data_loader_2)

203

In [78]:
list(iter(data_loader_2))[-1]

[tensor([[-0.1337,  1.1331],
         [ 0.8557, -0.0482],
         [ 0.8504, -0.0305],
         [ 1.0401,  1.1743]]),
 tensor([1, 1, 1, 0])]

#### Drop last
Parametr powodujący pominięcie ostatniego wsadu (batch):

In [79]:
data_loader_2 = data.DataLoader(dataset,batch_size=7, shuffle=True, drop_last=True)
next(iter(data_loader_2))[0].shape[0]*len(data_loader_2)

196

## Funkcje straty
Na poprzednich zajęciach obliczaliśmy funkcję straty ręcznie wykonując operacje na tensorach. Jest to w pełni poprawne rozwiązanie, ale możemy też nie wymyślać koła na nowo, ale wykorzystać wbudowane moduły. Pełna lista dostępna tutaj: https://pytorch.org/docs/stable/nn.html#loss-functions

In [80]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib notebook

In [81]:
tensor_1 = torch.arange(5.)
tensor_2 = torch.ones(5)
print(tensor_1, tensor_2)

tensor([0., 1., 2., 3., 4.]) tensor([1., 1., 1., 1., 1.])


### Błąd średniokwadratowy (Mean Squared Error) lub L2Loss

In [82]:
loss = nn.MSELoss()
loss(tensor_1,tensor_2)

tensor(3.)

$ \quad l_{n}=\left(x_{n}-y_{n}\right)^{2}$

In [83]:
loss = nn.MSELoss(reduction="none")
print(f"No reduction: {loss(tensor_1,tensor_2)}")
loss = nn.MSELoss(reduction="mean")
print(f"Mean: {loss(tensor_1,tensor_2)}")
loss = nn.MSELoss(reduction="sum")
print(f"Sum: {loss(tensor_1,tensor_2)}")

No reduction: tensor([1., 0., 1., 4., 9.])
Mean: 3.0
Sum: 15.0


In [84]:
F.mse_loss(tensor_1, tensor_2)

tensor(3.)

In [85]:
loss = nn.MSELoss(reduction="none")
x, y = torch.meshgrid(torch.linspace(0,10,100),torch.linspace(0,10,100))
z = loss(x,y)

fig = plt.figure()
ax = fig.gca(projection='3d')

ax.plot_trisurf(x.numpy().flatten(), y.numpy().flatten(), z.numpy().flatten(), linewidth=0.2, antialiased=True)

plt.show()

<IPython.core.display.Javascript object>

/var/folders/_8/8qs28kr50_v5rng3_4jhcdfr0000gn/T/ipykernel_31030/1952393758.py:6: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  ax = fig.gca(projection='3d')


### Błąd bezwzględny (Mean absolute Error) lub L1Loss

$ \quad l_{n}=\left|x_{n}-y_{n}\right|$

In [86]:
print(tensor_1, tensor_2)
loss = nn.L1Loss()
loss(tensor_1,tensor_2)

tensor([0., 1., 2., 3., 4.]) tensor([1., 1., 1., 1., 1.])


tensor(1.4000)

In [87]:
loss = nn.L1Loss(reduction="none")
print(f"No reduction: {loss(tensor_1,tensor_2)}")
loss = nn.L1Loss(reduction="mean")
print(f"Mean: {loss(tensor_1,tensor_2)}")
loss = nn.L1Loss(reduction="sum")
print(f"Sum: {loss(tensor_1,tensor_2)}")

No reduction: tensor([1., 0., 1., 2., 3.])
Mean: 1.399999976158142
Sum: 7.0


In [88]:
loss = nn.L1Loss(reduction="none")
x, y = torch.meshgrid(torch.linspace(0,10,100),torch.linspace(0,10,100))
z = loss(x,y)

fig = plt.figure()
ax = fig.gca(projection='3d')

ax.plot_trisurf(x.numpy().flatten(), y.numpy().flatten(), z.numpy().flatten(), linewidth=0.2, antialiased=True)

plt.show()

<IPython.core.display.Javascript object>

/var/folders/_8/8qs28kr50_v5rng3_4jhcdfr0000gn/T/ipykernel_31030/3911627702.py:6: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  ax = fig.gca(projection='3d')


### Entropia krzyżowa:
$l_{n}=-\left[y_{n} \cdot \log x_{n}+\left(1-y_{n}\right) \cdot \log \left(1-x_{n}\right)\right]$

In [89]:
tensor_1 = torch.rand(5)
tensor_2 = torch.ones(5)
print(tensor_1, tensor_2)
loss = nn.BCELoss(reduction="none")
loss(tensor_1,tensor_2)

tensor([0.8507, 0.0950, 0.1356, 0.1201, 0.0545]) tensor([1., 1., 1., 1., 1.])


tensor([0.1617, 2.3536, 1.9981, 2.1198, 2.9099])

In [90]:
loss(tensor_1*10,tensor_2)

RuntimeError: all elements of input should be between 0 and 1

In [91]:
loss = nn.BCEWithLogitsLoss(reduction="none")
loss(tensor_1*10,tensor_2)

tensor([2.0204e-04, 3.2687e-01, 2.2928e-01, 2.6314e-01, 4.5739e-01])

In [92]:
loss = nn.BCEWithLogitsLoss(reduction="none")
x, y = torch.meshgrid(torch.linspace(-10,10,100),torch.linspace(-10,10,100))
z = loss(x,y)

fig = plt.figure()
ax = fig.gca(projection='3d')

ax.plot_trisurf(x.numpy().flatten(), y.numpy().flatten(), z.numpy().flatten(), linewidth=0.2, antialiased=True)

plt.show()

<IPython.core.display.Javascript object>

/var/folders/_8/8qs28kr50_v5rng3_4jhcdfr0000gn/T/ipykernel_31030/2633775187.py:6: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  ax = fig.gca(projection='3d')


# Optymalizacja parametrów
W przykładowym problemie optymalizacji parametrów regresji ręcznie aktualizowaliśmy gradienty. W torch są do tego narzędzia zwane optymalizatorami, znajdują się one w pakiecie torch.optim.

Najprostszym optymalizatorem jest algorytm stochastycznego spadku gradientu (Stochastic Gradient Descent) - SGD

In [93]:
model = SimpleClassifier(num_inputs=2, num_hidden=4, num_outputs=1)
print(model)

SimpleClassifier(
  (linear1): Linear(in_features=2, out_features=4, bias=True)
  (act_fn): Tanh()
  (linear2): Linear(in_features=4, out_features=1, bias=True)
)


In [94]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

## Połączmy to wszystko razem

In [95]:
train_dataset = XORDataset(size=1000)
train_data_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True)

In [96]:
model = SimpleClassifier(num_inputs=2, num_hidden=4, num_outputs=1)
model.to(device)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

In [97]:
loss_module = nn.BCEWithLogitsLoss()

In [98]:
model.train() 

# Training loop
for epoch in range(100):
    for data_inputs, data_labels in data_loader:

        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
#         data_inputs = data_inputs.to(device)
#         data_labels = data_labels.to(device)

        ## Step 2: Run the model on the input data
        preds = model(data_inputs)
        preds = preds.squeeze(dim=1) # Output is [Batch size, 1], but we want [Batch size]

        ## Step 3: Calculate the loss
        loss = loss_module(preds, data_labels.float())

        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero. 
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad() 
        # Perform backpropagation
        loss.backward()

        ## Step 5: Update the parameters
        optimizer.step()
    print(f"Epoch: {epoch}, loss: {loss.item():.3}")

Epoch: 0, loss: 0.697
Epoch: 1, loss: 0.723
Epoch: 2, loss: 0.697
Epoch: 3, loss: 0.664
Epoch: 4, loss: 0.68
Epoch: 5, loss: 0.685
Epoch: 6, loss: 0.696
Epoch: 7, loss: 0.684
Epoch: 8, loss: 0.702
Epoch: 9, loss: 0.714
Epoch: 10, loss: 0.701
Epoch: 11, loss: 0.696
Epoch: 12, loss: 0.701
Epoch: 13, loss: 0.692
Epoch: 14, loss: 0.694
Epoch: 15, loss: 0.677
Epoch: 16, loss: 0.684
Epoch: 17, loss: 0.695
Epoch: 18, loss: 0.701
Epoch: 19, loss: 0.7
Epoch: 20, loss: 0.696
Epoch: 21, loss: 0.675
Epoch: 22, loss: 0.689
Epoch: 23, loss: 0.689
Epoch: 24, loss: 0.689
Epoch: 25, loss: 0.692
Epoch: 26, loss: 0.676
Epoch: 27, loss: 0.679
Epoch: 28, loss: 0.693
Epoch: 29, loss: 0.699
Epoch: 30, loss: 0.691
Epoch: 31, loss: 0.703
Epoch: 32, loss: 0.689
Epoch: 33, loss: 0.699
Epoch: 34, loss: 0.691
Epoch: 35, loss: 0.726
Epoch: 36, loss: 0.686
Epoch: 37, loss: 0.708
Epoch: 38, loss: 0.695
Epoch: 39, loss: 0.716
Epoch: 40, loss: 0.696
Epoch: 41, loss: 0.699
Epoch: 42, loss: 0.685
Epoch: 43, loss: 0.688
E

### Ewaluacja

In [99]:
test_dataset = XORDataset(size=500)
# drop_last -> Don't drop the last batch although it is smaller than 128
test_data_loader = data.DataLoader(test_dataset, batch_size=128, shuffle=False, drop_last=False)

In [100]:
model.eval() # Set model to eval mode
true_preds, num_preds = 0., 0.

with torch.no_grad(): # Deactivate gradients for the following code
    for data_inputs, data_labels in data_loader:

        # Determine prediction of model on dev set
        data_inputs, data_labels = data_inputs.to(device), data_labels.to(device)
        preds = model(data_inputs)
        preds = preds.squeeze(dim=1)
        preds = torch.sigmoid(preds) # Sigmoid to map predictions between 0 and 1
        pred_labels = (preds >= 0.5).long() # Binarize predictions to 0 and 1

        # Keep records of predictions for the accuracy metric (true_preds=TP+TN, num_preds=TP+TN+FP+FN)
        true_preds += (pred_labels == data_labels).sum()
        num_preds += data_labels.shape[0]

acc = true_preds / num_preds
print(f"Accuracy of the model: {100.0*acc:4.2f}%")

AssertionError: Torch not compiled with CUDA enabled

In [101]:
from matplotlib.colors import to_rgba

In [102]:
with torch.no_grad():
    data = test_dataset.data.cpu().numpy()
    label = test_dataset.label.cpu().numpy()
    data_0 = data[label == 0]
    data_1 = data[label == 1]

    plt.figure(figsize=(4,4))
    plt.scatter(data_0[:,0], data_0[:,1], edgecolor="#333", label="Class 0")
    plt.scatter(data_1[:,0], data_1[:,1], edgecolor="#333", label="Class 1")
    plt.title("Dataset samples")
    plt.ylabel(r"$x_2$")
    plt.xlabel(r"$x_1$")
    plt.legend()

    # Let's make use of a lot of operations we have learned above
    model.to(device)
    c0 = torch.Tensor(to_rgba("C0")).to(device)
    c1 = torch.Tensor(to_rgba("C1")).to(device)
    x1 = torch.arange(-0.5, 1.5, step=0.01, device=device)
    x2 = torch.arange(-0.5, 1.5, step=0.01, device=device)
    xx1, xx2 = torch.meshgrid(x1, x2) # Meshgrid function as in numpy
    model_inputs = torch.stack([xx1, xx2], dim=-1)
    preds = model(model_inputs)
    preds = torch.sigmoid(preds)
    output_image = preds * c0[None,None] + (1 - preds) * c1[None,None] # Specifying "None" in a dimension creates a new one
    output_image = output_image.cpu().numpy() # Convert to numpy array. This only works for tensors on CPU, hence first push to CPU
    plt.imshow(output_image, origin='upper', extent=(-0.5, 1.5, -0.5, 1.5))
    plt.grid(False)

<IPython.core.display.Javascript object>

AssertionError: Torch not compiled with CUDA enabled

## Zapisywanie i wczytywanie modelu

In [103]:
state_dict = model.state_dict()
print(state_dict)

OrderedDict([('linear1.weight', tensor([[-0.2816,  0.1665],
        [-0.0682,  0.3849],
        [-0.1675,  0.2550],
        [ 0.6766,  0.5167]])), ('linear1.bias', tensor([ 0.0521,  0.6578,  0.6331, -0.4581])), ('linear2.weight', tensor([[0.2352, 0.0309, 0.0145, 0.1328]])), ('linear2.bias', tensor([-0.0157]))])


In [104]:
torch.save(state_dict, "simple_model.tar")

In [105]:
# Load state dict from the disk (make sure it is the same name as above)
state_dict = torch.load("simple_model.tar")

# Create a new model and load the state
new_model = SimpleClassifier(num_inputs=2, num_hidden=4, num_outputs=1)
new_model.load_state_dict(state_dict)

# Verify that the parameters are the same
print("Original model\n", model.cpu().state_dict())
print("\nLoaded model\n", new_model.state_dict())

Original model
 OrderedDict([('linear1.weight', tensor([[-0.2816,  0.1665],
        [-0.0682,  0.3849],
        [-0.1675,  0.2550],
        [ 0.6766,  0.5167]])), ('linear1.bias', tensor([ 0.0521,  0.6578,  0.6331, -0.4581])), ('linear2.weight', tensor([[0.2352, 0.0309, 0.0145, 0.1328]])), ('linear2.bias', tensor([-0.0157]))])

Loaded model
 OrderedDict([('linear1.weight', tensor([[-0.2816,  0.1665],
        [-0.0682,  0.3849],
        [-0.1675,  0.2550],
        [ 0.6766,  0.5167]])), ('linear1.bias', tensor([ 0.0521,  0.6578,  0.6331, -0.4581])), ('linear2.weight', tensor([[0.2352, 0.0309, 0.0145, 0.1328]])), ('linear2.bias', tensor([-0.0157]))])


## Zrób to sam:

Spróbujmy przewidzieć ocenę wina na podstawie jego parametrów

In [106]:
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', delimiter=";")

#### ... Jakieś wstępne przetwarzanie danych?

In [107]:
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

In [108]:
import torch.utils.data as data

In [109]:
train_dataset = data.TensorDataset(torch.from_numpy(train.values[:,:-1]),torch.from_numpy(train.values[:,-1]))
next(iter(train_dataset))

(tensor([7.6000e+00, 3.0000e-01, 3.7000e-01, 1.6000e+00, 8.7000e-02, 2.7000e+01,
         1.7700e+02, 9.9438e-01, 3.0900e+00, 5.0000e-01, 9.8000e+00],
        dtype=torch.float64),
 tensor(5., dtype=torch.float64))